In [0]:
!pip install --upgrade pip

  Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


Instalando o gdal manualmente

In [0]:
%sh
#!/bin/bash
sudo apt-get update
# Isso instala o gdal na máquina
sudo apt-get install -y gdal-bin libgdal-dev
sudo apt-get install -y libproj-dev

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Fetched 114 kB in 1s (93.6 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (3.0.4+dfsg-1build3).
libgdal-dev is already the newest version (3.0.4+dfsg-1build3).
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
libproj-dev is already the newest version (6.3.1-1).
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.


os arquivos precisam estar com os nomes corretos

In [0]:
%sh
cd /usr/lib/python3/dist-packages/osgeo
mv _gdal.cpython-38-x86_64-linux-gnu.so _gdal.so
mv _gdalconst.cpython-38-x86_64-linux-gnu.so _gdalconst.so
mv _ogr.cpython-38-x86_64-linux-gnu.so _ogr.so
mv _osr.cpython-38-x86_64-linux-gnu.so _osr.so
mv _gdal_array.cpython-38-x86_64-linux-gnu.so _gdal_array.so
ls /usr/lib/python3/dist-packages/osgeo

mv: cannot stat '_gdal.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_gdalconst.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_ogr.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_osr.cpython-38-x86_64-linux-gnu.so': No such file or directory
mv: cannot stat '_gdal_array.cpython-38-x86_64-linux-gnu.so': No such file or directory
__init__.py
__pycache__
_gdal.so
_gdal_array.so
_gdalconst.so
_gnm.cpython-38-x86_64-linux-gnu.so
_ogr.so
_osr.so
gdal.py
gdal_array.py
gdalconst.py
gdalnumeric.py
gnm.py
ogr.py
osr.py


Bibliotecas

In [0]:
from pyspark.sql import SparkSession
import os

In [0]:
!pip install pyspark raster2xyz

  Using cached pyspark-3.5.1-py2.py3-none-any.whl
  Using cached raster2xyz-0.1.3-py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)


Iniciando sessão spark

In [0]:
spark=SparkSession.builder \
    .master('local[*]') \
    .getOrCreate()

Função para avaliar o benchmark

In [0]:
def benchmark(arquivo):
    df = spark.read.format("csv").option("header", True).load(arquivo)
    df.write.format("noop").mode("overwrite").save()

In [0]:
def partitions(arquivo):
  df = spark.read.format("csv").option("header", True).load(arquivo)
  print(f"Número de partições -> {df.rdd.getNumPartitions()}")

#Temperatura máxima

##10 minutos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tmax.zip
!unzip -o wc2.1_10m_tmax.zip

--2024-05-04 21:30:08--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tmax.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tmax.zip [following]
--2024-05-04 21:30:08--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_10m_tmax.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37446442 (36M) [application/zip]
Saving to: ‘wc2.1_10m_tmax.zip.1’

wc2.1_10m_tmax.zip. 100%[===================>]  35.71M  31.8MB/s    in 1.1s    

2024-05-04 21:30:10 (31.8 MB/s) - ‘wc2.1_10m_tmax.zip.1’ saved [37446442/37446442]

Archive:  wc2.1_10m_tmax.zip
  

In [0]:
!raster2xyz wc2.1_10m_tmax_01.tif wc2.1_10m_tmax_01.csv

[2024-05-04 09:30:12 - INFO] - Getting geotransform and data...
[2024-05-04 09:30:12 - INFO] - Getting XYZ data...
[2024-05-04 09:30:13 - INFO] - Getting geotransformed coordinates...
[2024-05-04 09:30:13 - INFO] - Building XYZ data...
[2024-05-04 09:30:22 - INFO] - New XYZ (csv file) created...


Os arquivos são salvos no cluster então é necessário movê-los para o dbfs

In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_10m_tmax_01.csv", "dbfs:/FileStore/tables/wc2.1_10m_tmax_01.csv")

Out[25]: True

Testando o benchmark

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_10m_tmax_01.csv'

In [0]:
partitions(arquivo)

Número de partições -> 8


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 7.25 ms, sys: 0 ns, total: 7.25 ms
Wall time: 5.78 s
CPU times: user 5.14 ms, sys: 4.24 ms, total: 9.38 ms
Wall time: 6.71 s
CPU times: user 7.52 ms, sys: 1.48 ms, total: 9 ms
Wall time: 5.77 s
CPU times: user 8.36 ms, sys: 916 µs, total: 9.28 ms
Wall time: 5.79 s
CPU times: user 4.48 ms, sys: 4.27 ms, total: 8.75 ms
Wall time: 5.95 s
CPU times: user 13.3 ms, sys: 602 µs, total: 13.9 ms
Wall time: 5.78 s
CPU times: user 4.93 ms, sys: 3.99 ms, total: 8.92 ms
Wall time: 5.37 s
CPU times: user 1.06 ms, sys: 7.48 ms, total: 8.54 ms
Wall time: 5.24 s
CPU times: user 6.2 ms, sys: 2.86 ms, total: 9.06 ms
Wall time: 5.38 s
CPU times: user 8.64 ms, sys: 117 µs, total: 8.76 ms
Wall time: 5.12 s
CPU times: user 8.86 ms, sys: 0 ns, total: 8.86 ms
Wall time: 5.66 s
CPU times: user 5.77 ms, sys: 3.09 ms, total: 8.86 ms
Wall time: 5.62 s
CPU times: user 0 ns, sys: 8.66 ms, total: 8.66 ms
Wall time: 5.4 s
CPU times: user 2.89 ms, sys: 5.51 ms, total: 8.4 ms
Wall time: 5.23 s
CPU times:

##5 minutos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tmax.zip
!unzip -o wc2.1_5m_tmax.zip

--2024-05-04 21:33:28--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tmax.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tmax.zip [following]
--2024-05-04 21:33:28--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_5m_tmax.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126715801 (121M) [application/zip]
Saving to: ‘wc2.1_5m_tmax.zip.1’

wc2.1_5m_tmax.zip.1 100%[===================>] 120.84M  42.3MB/s    in 2.9s    

2024-05-04 21:33:31 (42.3 MB/s) - ‘wc2.1_5m_tmax.zip.1’ saved [126715801/126715801]

Archive:  wc2.1_5m_tmax.zip
  in

In [0]:
!raster2xyz wc2.1_5m_tmax_01.tif wc2.1_5m_tmax_01.csv

[2024-05-04 09:33:35 - INFO] - Getting geotransform and data...
[2024-05-04 09:33:35 - INFO] - Getting XYZ data...
[2024-05-04 09:33:35 - INFO] - Getting geotransformed coordinates...
[2024-05-04 09:33:36 - INFO] - Building XYZ data...
[2024-05-04 09:34:14 - INFO] - New XYZ (csv file) created...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_5m_tmax_01.csv", "dbfs:/FileStore/tables/wc2.1_5m_tmax_01.csv")

Out[31]: True

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_5m_tmax_01.csv'

In [0]:
partitions(arquivo)

Número de partições -> 8


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 13.1 ms, sys: 5.46 ms, total: 18.6 ms
Wall time: 19.1 s
CPU times: user 9.65 ms, sys: 5.9 ms, total: 15.6 ms
Wall time: 19.6 s
CPU times: user 9.48 ms, sys: 5.1 ms, total: 14.6 ms
Wall time: 19.3 s
CPU times: user 14.8 ms, sys: 539 µs, total: 15.3 ms
Wall time: 19.7 s
CPU times: user 13.7 ms, sys: 2.75 ms, total: 16.4 ms
Wall time: 20.9 s
CPU times: user 11 ms, sys: 3.68 ms, total: 14.7 ms
Wall time: 16.8 s
CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 16.5 s
CPU times: user 13.4 ms, sys: 178 µs, total: 13.6 ms
Wall time: 17.2 s
CPU times: user 12 ms, sys: 1.72 ms, total: 13.7 ms
Wall time: 16.8 s
CPU times: user 9.69 ms, sys: 3.56 ms, total: 13.2 ms
Wall time: 15.7 s
CPU times: user 13.8 ms, sys: 337 µs, total: 14.1 ms
Wall time: 16.3 s
CPU times: user 19 ms, sys: 2.24 ms, total: 21.3 ms
Wall time: 16.7 s
CPU times: user 9.06 ms, sys: 4.22 ms, total: 13.3 ms
Wall time: 15.9 s
CPU times: user 11.7 ms, sys: 3.31 ms, total: 15 ms
Wall time: 17.4 s
CPU time

##2.5

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tmax.zip
!unzip -o wc2.1_2.5m_tmax.zip

--2024-05-04 21:55:34--  https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tmax.zip
Resolving biogeo.ucdavis.edu (biogeo.ucdavis.edu)... 128.120.228.172
Connecting to biogeo.ucdavis.edu (biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tmax.zip [following]
--2024-05-04 21:55:34--  https://data.biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_tmax.zip
Resolving data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)... 128.120.228.172
Connecting to data.biogeo.ucdavis.edu (data.biogeo.ucdavis.edu)|128.120.228.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444356872 (424M) [application/zip]
Saving to: ‘wc2.1_2.5m_tmax.zip.10’

wc2.1_2.5m_tmax.zip 100%[===================>] 423.77M  37.5MB/s    in 12s     

2024-05-04 21:55:46 (34.9 MB/s) - ‘wc2.1_2.5m_tmax.zip.10’ saved [444356872/444356872]

Archive:  wc2.1_2.5m

In [0]:
!raster2xyz wc2.1_2.5m_tmax_01.tif wc2.1_2.5m_tmax_01.csv

[2024-05-04 09:56:03 - INFO] - Getting geotransform and data...
[2024-05-04 09:56:04 - INFO] - Getting XYZ data...
[2024-05-04 09:56:06 - INFO] - Getting geotransformed coordinates...
[2024-05-04 09:56:08 - INFO] - Building XYZ data...
[2024-05-04 09:58:45 - INFO] - New XYZ (csv file) created...


In [0]:
dbutils.fs.cp("file:/databricks/driver/wc2.1_2.5m_tmax_01.csv", "dbfs:/FileStore/tables/wc2.1_2.5m_tmax_01.csv")

Out[48]: True

In [0]:
arquivo='dbfs:/FileStore/tables/wc2.1_2.5m_tmax_01.csv'


In [0]:
partitions(arquivo)

Número de partições -> 11


In [0]:
for i in range(1,33):
 %time benchmark(arquivo)

CPU times: user 22.8 ms, sys: 11 ms, total: 33.8 ms
Wall time: 1min 2s
CPU times: user 30.5 ms, sys: 6.08 ms, total: 36.6 ms
Wall time: 59.7 s
CPU times: user 27.8 ms, sys: 7.25 ms, total: 35.1 ms
Wall time: 1min 1s
CPU times: user 24.3 ms, sys: 14.5 ms, total: 38.8 ms
Wall time: 59.8 s
CPU times: user 29.1 ms, sys: 5.04 ms, total: 34.1 ms
Wall time: 59.7 s
CPU times: user 23.2 ms, sys: 13.5 ms, total: 36.8 ms
Wall time: 1min 3s
CPU times: user 17.8 ms, sys: 16.4 ms, total: 34.3 ms
Wall time: 1min
CPU times: user 24.8 ms, sys: 8.19 ms, total: 33 ms
Wall time: 1min
CPU times: user 24.5 ms, sys: 8.95 ms, total: 33.5 ms
Wall time: 1min 1s
CPU times: user 26.2 ms, sys: 11 ms, total: 37.2 ms
Wall time: 1min 1s
CPU times: user 18.6 ms, sys: 15 ms, total: 33.6 ms
Wall time: 1min
CPU times: user 17.2 ms, sys: 16.3 ms, total: 33.5 ms
Wall time: 1min
CPU times: user 18.7 ms, sys: 20 ms, total: 38.7 ms
Wall time: 1min
CPU times: user 21.2 ms, sys: 14.9 ms, total: 36 ms
Wall time: 59.9 s
CPU times

##30 segundos

In [0]:
!wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_tmax.zip
!unzip -o wc2.1_30s_tmax.zip

In [0]:
#arquivo mt grande raster não conseguiu rodar
!raster2xyz wc2.1_30s_prec_01.tif wc2.1_30s_prec_01.csv